In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
animals_df = pd.read_csv('/content/animals_dataset.csv')
persons_df = pd.read_csv('/content/persons_dataset.csv')

In [5]:
print("Animals Dataset:")
print(animals_df.head())

Animals Dataset:
   Animal_ID Type       Breed    Size  Age Good_with_Cats Good_with_Dogs  \
0          1  Cat  Maine Coon  Medium    9             No            Yes   
1          2  Dog     Siamese   Large    2            Yes            Yes   
2          3  Cat     Bulldog   Small   11            Yes             No   
3          4  Dog    Labrador  Medium    5            Yes            Yes   
4          5  Dog  Maine Coon   Small   14            Yes            Yes   

  Apartment_Friendly  
0                Yes  
1                Yes  
2                 No  
3                 No  
4                 No  


In [6]:
print("\nPersons Dataset:")
print(persons_df.head())


Persons Dataset:
   Person_ID Preferred_Type Preferred_Size Other_Cats Other_Dogs  \
0          1            Dog          Small        Yes        Yes   
1          2            Cat          Large        Yes        Yes   
2          3            Dog          Small        Yes        Yes   
3          4            Dog          Large         No        Yes   
4          5            Dog          Large        Yes         No   

  Lives_in_Apartment  
0                 No  
1                 No  
2                Yes  
3                 No  
4                Yes  


In [8]:
animals_df['features'] = animals_df[['Type', 'Breed', 'Size', 'Age', 'Good_with_Cats', 'Good_with_Dogs', 'Apartment_Friendly']].astype(str).agg(' '.join, axis=1)
persons_df['preferences'] = persons_df[['Preferred_Type', 'Preferred_Size', 'Other_Cats', 'Other_Dogs', 'Lives_in_Apartment']].astype(str).agg(' '.join, axis=1)



In [9]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

animals_tfidf_matrix = tfidf_vectorizer.fit_transform(animals_df['features'])
persons_tfidf_matrix = tfidf_vectorizer.transform(persons_df['preferences'])

In [10]:
cosine_sim = cosine_similarity(persons_tfidf_matrix, animals_tfidf_matrix)

In [11]:
def get_recommendations(cosine_sim_matrix, animal_ids, top_n=5):
    recommendations = {}
    for idx, row in enumerate(cosine_sim_matrix):
        similar_indices = row.argsort()[-top_n:][::-1]
        similar_items = [(animal_ids[i], row[i]) for i in similar_indices]
        recommendations[idx] = similar_items
    return recommendations


In [12]:
animal_ids = animals_df['Animal_ID'].tolist()
recommendations = get_recommendations(cosine_sim, animal_ids)

In [13]:
for person_idx, recs in recommendations.items():
    print(f"\nRecommendations for Person {persons_df.iloc[person_idx]['Person_ID']}:")
    for animal_id, score in recs:
        print(f"Animal ID: {animal_id}, Similarity Score: {score:.4f}")


Görüntülenen çıkış son 5000 satıra kısaltıldı.
Animal ID: 272, Similarity Score: 0.6895
Animal ID: 594, Similarity Score: 0.6895

Recommendations for Person 287:
Animal ID: 818, Similarity Score: 0.8201
Animal ID: 336, Similarity Score: 0.8201
Animal ID: 962, Similarity Score: 0.8201
Animal ID: 665, Similarity Score: 0.8201
Animal ID: 2, Similarity Score: 0.8163

Recommendations for Person 288:
Animal ID: 470, Similarity Score: 0.7441
Animal ID: 942, Similarity Score: 0.7441
Animal ID: 994, Similarity Score: 0.7441
Animal ID: 395, Similarity Score: 0.7441
Animal ID: 435, Similarity Score: 0.7441

Recommendations for Person 289:
Animal ID: 818, Similarity Score: 0.8201
Animal ID: 336, Similarity Score: 0.8201
Animal ID: 962, Similarity Score: 0.8201
Animal ID: 665, Similarity Score: 0.8201
Animal ID: 2, Similarity Score: 0.8163

Recommendations for Person 290:
Animal ID: 283, Similarity Score: 0.7478
Animal ID: 860, Similarity Score: 0.7478
Animal ID: 894, Similarity Score: 0.7478
Anima

In [14]:
def test_recommendation(new_person_data, top_n=5):
    new_person_df = pd.DataFrame([new_person_data])
    new_person_df['preferences'] = new_person_df[['Preferred_Type', 'Preferred_Size', 'Other_Cats', 'Other_Dogs', 'Lives_in_Apartment']].astype(str).agg(' '.join, axis=1)
    new_person_tfidf_matrix = tfidf_vectorizer.transform(new_person_df['preferences'])

    new_cosine_sim = cosine_similarity(new_person_tfidf_matrix, animals_tfidf_matrix)

    new_recommendations = get_recommendations(new_cosine_sim, animal_ids, top_n)

    print(f"\nRecommendations for new person:")
    for recs in new_recommendations.values():
        for animal_id, score in recs:
            print(f"Animal ID: {animal_id}, Similarity Score: {score:.4f}")


In [16]:
new_person_data_1 = {
    'Preferred_Type': 'Dog',
    'Preferred_Size': 'Medium',
    'Other_Cats': 'No',
    'Other_Dogs': 'Yes',
    'Lives_in_Apartment': 'Yes'
}

new_person_data_2 = {
    'Preferred_Type': 'Cat',
    'Preferred_Size': 'Large',
    'Other_Cats': 'Yes',
    'Other_Dogs': 'No',
    'Lives_in_Apartment': 'No'
}

new_person_data_3 = {
    'Preferred_Type': 'Dog',
    'Preferred_Size': 'Small',
    'Other_Cats': 'Yes',
    'Other_Dogs': 'Yes',
    'Lives_in_Apartment': 'Yes'
}


In [17]:
print("Test Case 1:")
test_recommendation(new_person_data_1)

print("\nTest Case 2:")
test_recommendation(new_person_data_2)

print("\nTest Case 3:")
test_recommendation(new_person_data_3)

Test Case 1:

Recommendations for new person:
Animal ID: 648, Similarity Score: 0.8211
Animal ID: 834, Similarity Score: 0.8172
Animal ID: 163, Similarity Score: 0.8172
Animal ID: 697, Similarity Score: 0.8172
Animal ID: 59, Similarity Score: 0.8120

Test Case 2:

Recommendations for new person:
Animal ID: 443, Similarity Score: 0.7460
Animal ID: 87, Similarity Score: 0.7460
Animal ID: 937, Similarity Score: 0.7460
Animal ID: 667, Similarity Score: 0.7460
Animal ID: 931, Similarity Score: 0.7460

Test Case 3:

Recommendations for new person:
Animal ID: 172, Similarity Score: 0.8379
Animal ID: 198, Similarity Score: 0.8379
Animal ID: 65, Similarity Score: 0.8379
Animal ID: 568, Similarity Score: 0.8379
Animal ID: 83, Similarity Score: 0.8379
